In [1]:
import argparse

import sys
import os

sys.path.append(os.environ['REPO_DIR'] + '/utilities')
from utilities2015 import *
from data_manager import *
from metadata import *
from distributed_utilities import *
from learning_utilities import *

from scipy.ndimage.interpolation import map_coordinates
from skimage.exposure import rescale_intensity, adjust_gamma
from skimage.transform import rotate


ENABLE_UPLOAD_S3 is not set, default to False.
ENABLE_DOWNLOAD_S3 is not set, default to False.
No vtk
Failed to cache MD585 section_limits: ini file /media/yuncong/BstemAtlasData/atlas_data/CSHL_data_processed/MD585/MD585_prep2_sectionLimits.ini does not exist.
Seems you are using operation INIs to provide cropbox.
Failed to cache MD585 cropbox: ini file /media/yuncong/BstemAtlasData/atlas_data/CSHL_data_processed/MD585/operation_configs/from_padded_to_brainstem.ini does not exist.
Failed to cache MD585 valid_sections/filenames: MD585
Seems you are using operation INIs to provide cropbox.
Failed to cache MD585 image_shape: ini file /media/yuncong/BstemAtlasData/atlas_data/CSHL_data_processed/MD585/operation_configs/from_padded_to_brainstem.ini does not exist.
No anchor.txt is found. Seems we are using the operation ini to provide anchor. Try to load operation ini.
Failed to cache MD594 anchor: ini file /media/yuncong/BstemAtlasData/atlas_data/CSHL_data_processed/MD594/operation_config

Setting environment for Precision WorkStation
{'MD585': 0.46}


/home/yuncong/MouseBrainAtlas/mousebrainatlas_virtualenv/local/lib/python2.7/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d
xgboost is not loaded.


In [2]:
stack = 'MD585'

In [8]:
section = 230

In [9]:
normalized_float_map = load_data(DataManager.get_intensity_normalization_result_filepath(what='normalized_float_map', stack=stack, section=section))

In [10]:
normalized_float_map

(25205, 32032)

In [11]:
gamma_map = img_as_ubyte(adjust_gamma(np.arange(0, 256, 1) / 255., 8.))

/home/yuncong/MouseBrainAtlas/mousebrainatlas_virtualenv/local/lib/python2.7/site-packages/skimage/util/dtype.py:122: UserWarning: Possible precision loss when converting from float64 to uint8
  .format(dtypeobj_in, dtypeobj_out))


In [14]:
metadata_cache['sections_to_filenames'][stack][230]

'MD585-IHC41-2015.08.19-14.21.09_MD585_1_0121'

In [15]:
image_name = 'MD585-IHC41-2015.08.19-14.21.09_MD585_1_0121'

In [16]:
img_normalized = load_data(
          DataManager.get_intensity_normalization_result_filepath(what='normalized_float_map', stack=stack, fn=image_name),
         download_s3=False)  

In [28]:
img_normalized_uint8 = rescale_intensity_v2(img_normalized, -5, 4)

In [29]:
display_image(img_normalized_uint8[::2, ::2])

/home/yuncong/MouseBrainAtlas/src/preprocess/tmp.jpg

In [ ]:
low = -2.
high = 50.

t = time.time()
img_normalized_uint8 = rescale_intensity_v2(img_normalized, low, high)
sys.stderr.write('Rescale to uint8: %.2f seconds.\n' % (time.time() - t))

t = time.time()
raw_mask = load_data(DataManager.get_image_filepath_v2(stack=stack, prep_id=prep_id, fn=image_name, version='mask', resol=resol, ext='bp'),
                    download_s3=False)
img_normalized_uint8[~raw_mask] = 0
sys.stderr.write('Load mask: %.2f seconds.\n' % (time.time() - t))

img = 255 - img_normalized_uint8
save_data(gamma_map[img], 
          DataManager.get_image_filepath_v2(stack=stack, prep_id=prep_id, fn=image_name, version=args.out_version, resol=resol),
         upload_s3=False)
